In [9]:
%matplotlib inline

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder

In [10]:
df = pd.read_csv('../../Data/model-data/model_data.csv', index_col = 0)

#replace independent with ind. for better graphs
df.loc[df['Team Conference'] == 'Independent', 'Team Conference'] = 'Ind.'
df.loc[df['Opponent Conference'] == 'Indepedent', 'Opponent Conference'] = 'Ind.'

df.head(5)

,Team,Opponent,syn_date,Year,Team Win,Team Conference,Opponent Conference,Team Win_3_game_average,Team Earned YPPA_3_game_average,Team Earned YPRA_3_game_average,...,Opp Coaches Compensation,Opp Athletic Student Aid,Opp Total Revenues,"Opp Corporate Sponsorship, Advertising, Licensing",Opp Donor Contributions,Opp Competition Guarantees,Opp Ticket Sales,Opp Total Football Spending,Opp footall_expense_pct,Opp football_expense_pct
0,Akron,Ohio State,2011-09-01,2011,0,MAC,Big 10,NaN,NaN,NaN,...,16511360.0,15129920.0,131815821.0,12162905.0,17636938.0,3853330.0,50009395.0,44287914.0,0.362164,0.362164
1,Alabama,Kent State,2011-09-01,2011,1,SEC,MAC,NaN,NaN,NaN,...,3763340.0,5287795.0,21448821.0,424323.0,435339.0,822050.0,669316.0,11774328.0,0.567253,0.567253
2,Arkansas State,Illinois,2011-09-01,2011,0,Sun Belt,Big 10,NaN,NaN,NaN,...,10259368.0,9847571.0,77863883.0,4553520.0,16437955.0,1376000.0,16533261.0,29526365.0,0.401846,0.401846
3,Army,Northern Illinois,2011-09-01,2011,0,Ind.,MAC,NaN,NaN,NaN,...,3593287.0,6810435.0,24148602.0,370107.0,1329546.0,1629150.0,864185.0,13076496.0,0.573691,0.573691
4,Auburn,Utah State,2011-09-01,2011,1,SEC,WAC,NaN,NaN,NaN,...,3887698.0,4389402.0,22777944.0,564983.0,1869811.0,831640.0,1965806.0,11704878.0,0.594142,0.594142
